# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

***
*WORK IN PROGRESS*
***

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hopspitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hopspitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'06-15-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'06-15-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-06-16 04:33:09,34.223334,-82.461707,70,0,0,70,"Abbeville, South Carolina, US",285.399764,0.000000
1,22001.0,Acadia,Louisiana,US,2020-06-16 04:33:09,30.295065,-92.414197,597,31,0,566,"Acadia, Louisiana, US",962.204851,5.192630
2,51001.0,Accomack,Virginia,US,2020-06-16 04:33:09,37.767072,-75.632346,996,14,0,982,"Accomack, Virginia, US",3082.064612,1.405622
3,16001.0,Ada,Idaho,US,2020-06-16 04:33:09,43.452658,-116.241552,892,22,0,870,"Ada, Idaho, US",185.220947,2.466368
4,19001.0,Adair,Iowa,US,2020-06-16 04:33:09,41.330756,-94.471059,11,0,0,11,"Adair, Iowa, US",153.803132,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until day is over.**

In [6]:
jh_cases_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_cases_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,19551,20342,20917,21459,22142,22890,23546,24102,24766,25527
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1232,1246,1263,1299,1341,1385,1416,1464,1521,1590
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,10050,10154,10265,10382,10484,10589,10698,10810,10919,11031


In [7]:
jh_deaths_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_deaths_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,327,357,369,384,405,426,446,451,471,478
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,34,34,34,34,34,35,36,36,36,36
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,698,707,715,724,732,741,751,760,767,777


In [8]:
jh_recovered_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_recovered_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1830,1875,2171,2651,3013,3326,3928,4201,4725,5164
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,925,938,945,960,980,1001,1034,1039,1044,1055
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,6631,6717,6799,6951,7074,7255,7322,7420,7606,7735


In [9]:
jh_cases_counties_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_cases_counties_hist.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,179,179,179,180,180,183,183,183,183,185
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,26,27,28,30,30,30,30,30,30,30


In [10]:
jh_deaths_counties_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_deaths_counties_hist.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [11]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
2483,1117.0,Shelby,Alabama,US,2020-06-16 04:33:09,33.268798,-86.662326,655,20,0,635,"Shelby, Alabama, US",300.869997,3.053435
197,1007.0,Bibb,Alabama,US,2020-06-16 04:33:09,32.996421,-87.125115,108,1,0,107,"Bibb, Alabama, US",482.272037,0.925926
2176,1109.0,Pike,Alabama,US,2020-06-16 04:33:09,31.803964,-85.940830,334,2,0,332,"Pike, Alabama, US",1008.636830,0.598802


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [12]:
jh_countries_live = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [13]:
jh_countries_live.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,2114026,116127,576334
Brazil,888271,43959,477709
Russia,536484,7081,284021


In [14]:
jh_countries_live['Active'] = jh_countries_live['Confirmed'] - jh_countries_live['Deaths'] - jh_countries_live['Recovered']

In [15]:
jh_countries_live['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_countries_live['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_countries_live['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [16]:
jh_countries_live.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,2114026,116127,576334,1421565,2020-06-16 04:33:09,38.253180,-91.417561
Brazil,888271,43959,477709,366603,2020-06-16 04:33:09,-12.669522,-48.480493
Russia,536484,7081,284021,245382,2020-06-16 04:33:09,54.546312,62.120860
India,343091,9900,180013,153178,2020-06-16 04:33:09,23.356511,82.081064
United Kingdom,298315,41821,1284,255210,2020-06-16 04:33:09,30.744028,-35.367255


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [17]:
jh_states_live = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [18]:
jh_states_live.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
New York,383944,30856,0
New Jersey,167103,12708,0
California,155726,5114,0


In [19]:
jh_states_live['Active'] = jh_states_live['Confirmed'] - jh_states_live['Deaths'] - jh_states_live['Recovered']

In [20]:
jh_states_live['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_states_live['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_states_live['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [21]:
jh_states_live.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
New York,383944,30856,0,353088,2020-06-16 04:33:09,42.671593,-75.579694
New Jersey,167103,12708,0,154395,2020-06-16 04:33:09,40.286441,-74.622450
California,155726,5114,0,150612,2020-06-16 04:33:09,37.778248,-120.728666
Illinois,133016,6326,0,126690,2020-06-16 04:33:09,39.843618,-89.178330
Massachusetts,105690,7647,0,98043,2020-06-16 04:33:09,42.066277,-71.418469


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [22]:
jh_counties_live = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_counties_live.sort_values('Confirmed', ascending=False).head(5)

Confirmed  Deaths  Recovered
Province_State Admin2                                     
New York       New York City     210259   22124          0
Illinois       Cook               85184    4206          0
California     Los Angeles        73832    2926          0
New York       Nassau             41240    2167          0
               Suffolk            40692    1955          0

In [24]:
jh_counties_live['Active'] = jh_counties_live['Confirmed'] - jh_counties_live['Deaths'] - jh_counties_live['Recovered']


In [25]:
jh_counties_live['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_counties_live['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_counties_live['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_counties_live['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [26]:
jh_counties_live.sort_values('Confirmed', ascending=False).head(5)

Confirmed  Deaths  Recovered  Active  \
Province_State Admin2                                                
New York       New York City     210259   22124          0  188135   
Illinois       Cook               85184    4206          0   80978   
California     Los Angeles        73832    2926          0   70906   
New York       Nassau             41240    2167          0   39073   
               Suffolk            40692    1955          0   38737   

                                      Last_Update        Lat        Long  \
Province_State Admin2                                                      
New York       New York City  2020-06-16 04:33:09  40.767273  -73.971526   
Illinois       Cook           2020-06-16 04:33:09  41.841448  -87.816588   
California     Los Angeles    2020-06-16 04:33:09  34.308284 -118.228241   
New York       Nassau         2020-06-16 04:33:09  40.740665  -73.589419   
               Suffolk        2020-06-16 04:33:09  40.883201  -72.801217   

                                 FIPS  
Province_State Admin2                  
New York       New York City  36061.0  
Illinois       Cook           17031.0  
California     Los Angeles     6037.0  
New York       Nassau         36059.0  
               Suffolk        36103.0

##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [27]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_cases_global_hist.iloc[:, 4:].columns

In [28]:
jh_cases_countries_hist_ts = pd.pivot_table(jh_cases_global_hist, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_cases_countries_hist_ts = jh_cases_countries_hist_ts.transpose()
jh_cases_countries_hist_ts.index = pd.to_datetime(jh_cases_countries_hist_ts.index)
jh_cases_countries_hist_ts = jh_cases_countries_hist_ts.sort_index()

In [29]:
jh_cases_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-13,24102,1464,10810,853,138,26,30295,16004,7320,17078,...,295828,847,4966,2904,334,489,9,705,1357,356
2020-06-14,24766,1521,10919,853,140,26,31577,16667,7335,17109,...,297342,848,5080,2978,334,492,9,728,1358,383
2020-06-15,25527,1590,11031,853,142,26,32785,17064,7347,17135,...,298315,848,5263,3062,334,505,9,844,1382,387


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [30]:
jh_deaths_countries_hist_ts = pd.pivot_table(jh_deaths_global_hist, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_deaths_countries_hist_ts = jh_deaths_countries_hist_ts.transpose()
jh_deaths_countries_hist_ts.index = pd.to_datetime(jh_deaths_countries_hist_ts.index)
jh_deaths_countries_hist_ts = jh_deaths_countries_hist_ts.sort_index()

In [31]:
jh_deaths_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-13,451,36,760,51,6,3,815,264,102,677,...,41747,23,19,24,0,3,1,160,10,4
2020-06-14,471,36,767,51,6,3,833,269,102,677,...,41783,23,19,25,0,3,1,164,11,4
2020-06-15,478,36,777,51,6,3,854,285,102,678,...,41821,23,19,26,0,3,1,208,11,4


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [32]:
jh_recovered_countries_hist_ts = pd.pivot_table(jh_recovered_global_hist, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_recovered_countries_hist_ts = jh_recovered_countries_hist_ts.transpose()
jh_recovered_countries_hist_ts.index = pd.to_datetime(jh_recovered_countries_hist_ts.index)
jh_recovered_countries_hist_ts = jh_recovered_countries_hist_ts.sort_index()

In [33]:
jh_recovered_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-13,4201,1039,7420,781,61,20,9083,6081,6838,16012,...,1283,784,3874,487,323,415,8,39,1104,54
2020-06-14,4725,1044,7606,781,61,20,9564,6214,6851,16059,...,1283,788,3943,835,323,415,8,53,1122,54
2020-06-15,5164,1055,7735,789,64,20,9891,6276,6856,16066,...,1284,792,4019,835,323,415,8,79,1142,54


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [34]:
jh_cases_counties_hist.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20
3147,84080001,US,USA,840,80001.0,Out of AL,Alabama,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
52,84001095,US,USA,840,1095.0,Marshall,Alabama,US,34.369760,-86.304867,...,730,739,744,752,764,786,811,849,882,896
51,84001093,US,USA,840,1093.0,Marion,Alabama,US,34.136974,-87.887042,...,126,126,126,126,126,129,132,134,140,142
50,84001091,US,USA,840,1091.0,Marengo,Alabama,US,32.247676,-87.787962,...,182,184,187,187,192,200,204,206,209,221
48,84001087,US,USA,840,1087.0,Macon,Alabama,US,32.387599,-85.692677,...,96,98,100,104,105,110,117,122,125,126


In [35]:
date_columns = jh_cases_counties_hist.iloc[:,11:].columns

In [36]:
jh_cases_states_hist_ts = pd.pivot_table(jh_cases_counties_hist, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_cases_states_hist_ts = jh_cases_states_hist_ts.transpose()
jh_cases_states_hist_ts.index = pd.to_datetime(jh_cases_states_hist_ts.index)
jh_cases_states_hist_ts = jh_cases_states_hist_ts.sort_index()

In [37]:
jh_cases_states_hist_ts.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-13,24601,653,0,34660,12095,150018,29002,44994,10229,49,...,29549,86915,13981,1125,72,53869,25538,2274,22518,1050
2020-06-14,25615,660,0,35706,12501,152300,29115,45088,10264,49,...,30343,88523,14313,1127,72,54506,25834,2290,22758,1060
2020-06-15,26272,663,0,36844,12917,155726,29284,45235,10340,49,...,31004,90211,14608,1128,72,54886,26158,2322,22932,1079


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [38]:
jh_cases_counties_hist_ts = pd.pivot_table(jh_cases_counties_hist, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_cases_counties_hist_ts = jh_cases_counties_hist_ts.transpose()
jh_cases_counties_hist_ts.index = pd.to_datetime(jh_cases_counties_hist_ts.index)
jh_cases_counties_hist_ts = jh_cases_counties_hist_ts.sort_index()

In [39]:
jh_cases_counties_hist_ts.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-13         331     361     226  100    102     276    517     190   
2020-06-14         357     364     234  104    110     302    536     195   
2020-06-15         364     368     237  108    114     305    541     196   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-13          438       47  ...       5      1       19        3   
2020-06-14          453       51  ...       5      1       19        3   
2020-06-15          461       51  ...       5      1       19        3   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-13             40   104    71          0       39      1  
2020-06-14             40   105    73          0       39      1  
2020-06-15             40   105    88          0       39      1  

[3 rows x 3254 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [40]:
jh_deaths_states_hist_ts = pd.pivot_table(jh_deaths_counties_hist, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_deaths_states_hist_ts = jh_deaths_states_hist_ts.transpose()
jh_deaths_states_hist_ts.index = pd.to_datetime(jh_deaths_states_hist_ts.index)
jh_deaths_states_hist_ts = jh_deaths_states_hist_ts.sort_index()

In [41]:
jh_deaths_states_hist_ts.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-13,773,12,0,1189,177,5076,1597,4186,419,0,...,472,1971,139,55,6,1541,1213,88,691,18
2020-06-14,773,12,0,1191,179,5099,1598,4201,422,0,...,474,1984,139,55,6,1546,1217,88,692,18
2020-06-15,774,12,0,1203,182,5114,1605,4204,423,0,...,482,2001,143,55,6,1552,1220,88,694,18


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [42]:
jh_deaths_counties_hist_ts = pd.pivot_table(jh_deaths_counties_hist, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_deaths_counties_hist_ts = jh_deaths_counties_hist_ts.transpose()
jh_deaths_counties_hist_ts.index = pd.to_datetime(jh_deaths_counties_hist_ts.index)
jh_deaths_counties_hist_ts = jh_deaths_counties_hist_ts.sort_index()

In [43]:
jh_deaths_counties_hist_ts.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-13           6       9       1    1      1       8     25       3   
2020-06-14           6       9       1    1      1       8     25       3   
2020-06-15           6       9       1    1      1       8     25       3   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-13           26        5  ...       0      0        0        0   
2020-06-14           26        5  ...       0      0        0        0   
2020-06-15           26        5  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-13              0     0     0         17        0      0  
2020-06-14              0     0     0         17        0      0  
2020-06-15              0     0     0         17        0      0  

[3 rows x 3254 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [44]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_countries_live.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [45]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_states_live.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [46]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_counties_live.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [47]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_cases_countries_hist_ts.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [48]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_deaths_countries_hist_ts.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_recovered_countries_hist_ts.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_cases_states_hist_ts.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_cases_counties_hist_ts.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_deaths_states_hist_ts.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_deaths_counties_hist_ts.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [54]:
jh_cases_countries_hist_pct = jh_cases_countries_hist_ts.pct_change()

In [55]:
jh_cases_countries_hist_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-13,0.023613,0.033898,0.010469,0.0,0.061538,0.0,0.053226,0.047314,0.003565,0.000820,...,0.004844,0.000000,0.019922,0.008684,0.003003,0.000000,0.0,0.115506,0.027252,0.037901
2020-06-14,0.027550,0.038934,0.010083,0.0,0.014493,0.0,0.042317,0.041427,0.002049,0.001815,...,0.005118,0.001181,0.022956,0.025482,0.000000,0.006135,0.0,0.032624,0.000737,0.075843
2020-06-15,0.030728,0.045365,0.010257,0.0,0.014286,0.0,0.038256,0.023820,0.001636,0.001520,...,0.003272,0.000000,0.036024,0.028207,0.000000,0.026423,0.0,0.159341,0.017673,0.010444


In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_cases_countries_hist_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [57]:
jh_deaths_countries_hist_pct = jh_deaths_countries_hist_ts.pct_change()

In [58]:
jh_deaths_countries_hist_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-13,0.011211,0.0,0.011984,0.0,0.2,0.0,0.038217,0.023256,0.0,0.002963,...,0.004355,0.0,0.0,0.043478,NaN,0.0,0.0,0.151079,0.0,0.0
2020-06-14,0.044346,0.0,0.009211,0.0,0.0,0.0,0.022086,0.018939,0.0,0.000000,...,0.000862,0.0,0.0,0.041667,NaN,0.0,0.0,0.025000,0.1,0.0
2020-06-15,0.014862,0.0,0.013038,0.0,0.0,0.0,0.025210,0.059480,0.0,0.001477,...,0.000909,0.0,0.0,0.040000,NaN,0.0,0.0,0.268293,0.0,0.0


In [59]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_deaths_countries_hist_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [60]:
jh_recovered_countries_hist_pct = jh_recovered_countries_hist_ts.pct_change()

In [61]:
jh_recovered_countries_hist_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-13,0.069501,0.004836,0.013384,0.000000,0.452381,0.0,0.038888,0.078383,0.005145,0.001689,...,0.000780,0.005128,0.030867,0.000000,0.0,0.002415,0.333333,0.392857,0.000000,0.058824
2020-06-14,0.124732,0.004812,0.025067,0.000000,0.000000,0.0,0.052956,0.021871,0.001901,0.002935,...,0.000000,0.005102,0.017811,0.714579,0.0,0.000000,0.000000,0.358974,0.016304,0.000000
2020-06-15,0.092910,0.010536,0.016960,0.010243,0.049180,0.0,0.034191,0.009977,0.000730,0.000436,...,0.000779,0.005076,0.019275,0.000000,0.0,0.000000,0.000000,0.490566,0.017825,0.000000


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_recovered_countries_hist_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [63]:
jh_cases_states_hist_pct = jh_cases_states_hist_ts.pct_change()

In [64]:
jh_cases_states_hist_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-13,0.037579,0.046474,NaN,0.049063,0.047458,0.022903,0.006769,0.006825,0.005505,0.0,...,0.014802,0.023408,0.029756,0.005362,0.0,0.012366,0.014580,0.011116,0.012227,0.022395
2020-06-14,0.041218,0.010720,NaN,0.030179,0.033568,0.015212,0.003896,0.002089,0.003422,0.0,...,0.026871,0.018501,0.023747,0.001778,0.0,0.011825,0.011591,0.007036,0.010658,0.009524
2020-06-15,0.025649,0.004545,NaN,0.031871,0.033277,0.022495,0.005805,0.003260,0.007405,0.0,...,0.021784,0.019068,0.020611,0.000887,0.0,0.006972,0.012542,0.013974,0.007646,0.017925


In [65]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_cases_states_hist_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [66]:
jh_cases_counties_hist_pct = jh_cases_counties_hist_ts.pct_change()

In [67]:
jh_cases_counties_hist_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-06-13      0.024768  0.022663  0.022624  0.030928  0.073684  0.069767   
2020-06-14      0.078550  0.008310  0.035398  0.040000  0.078431  0.094203   
2020-06-15      0.019608  0.010989  0.012821  0.038462  0.036364  0.009934   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-06-13      0.036072  0.010638  0.025761  0.021739  ...  0.666667    0.0   
2020-06-14      0.036750  0.026316  0.034247  0.085106  ...  0.000000    0.0   
2020-06-15      0.009328  0.005128  0.017660  0.000000  ...  0.000000    0.0   

Province_State                                                              \
Admin2         Sheridan Sublette Sweetwater     Teton     Uinta Unassigned   
2020-06-13          0.0      0.0   0.052632  0.000000  0.145161        NaN   
2020-06-14          0.0      0.0   0.000000  0.009615  0.028169        NaN   
2020-06-15          0.0      0.0   0.000000  0.000000  0.205479        NaN   

Province_State                  
Admin2         Washakie Weston  
2020-06-13          0.0    0.0  
2020-06-14          0.0    0.0  
2020-06-15          0.0    0.0  

[3 rows x 3254 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_cases_counties_hist_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_deaths_states_hist_pct = jh_deaths_states_hist_ts.pct_change()

In [70]:
jh_deaths_states_hist_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-13,0.005202,0.0,NaN,0.028547,0.005682,0.019687,0.009482,0.006492,0.012077,NaN,...,0.012876,0.007669,0.000000,0.0,0.0,0.004563,0.007475,0.0,0.002903,0.0
2020-06-14,0.000000,0.0,NaN,0.001682,0.011299,0.004531,0.000626,0.003583,0.007160,NaN,...,0.004237,0.006596,0.000000,0.0,0.0,0.003245,0.003298,0.0,0.001447,0.0
2020-06-15,0.001294,0.0,NaN,0.010076,0.016760,0.002942,0.004380,0.000714,0.002370,NaN,...,0.016878,0.008569,0.028777,0.0,0.0,0.003881,0.002465,0.0,0.002890,0.0


In [71]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_deaths_states_hist_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [72]:
jh_deaths_counties_hist_pct = jh_deaths_counties_hist_ts.pct_change()

In [73]:
jh_deaths_counties_hist_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-13         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2020-06-14         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2020-06-15         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-13          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-06-14          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-06-15          0.0      0.0  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-13            NaN   NaN   NaN        0.0      NaN    NaN  
2020-06-14            NaN   NaN   NaN        0.0      NaN    NaN  
2020-06-15            NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3254 columns]

In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_deaths_counties_hist_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [75]:
jh_cases_countries_hist_pct_ewma = jh_cases_countries_hist_pct.ewm(span=3).mean()

In [76]:
jh_cases_countries_hist_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-13,0.027059,0.029975,0.010369,0.000324,0.074684,2.441406e-06,0.051787,0.042992,0.002165,0.001263,...,0.004837,0.000174,0.022897,0.019459,0.002361,0.002782,2.384186e-07,0.088825,0.040431,0.036991
2020-06-14,0.027304,0.034455,0.010226,0.000162,0.044588,1.220703e-06,0.047052,0.042210,0.002107,0.001539,...,0.004977,0.000677,0.022926,0.022471,0.001181,0.004458,1.192093e-07,0.060724,0.020584,0.056417
2020-06-15,0.029016,0.039910,0.010242,0.000081,0.029437,6.103516e-07,0.042654,0.033015,0.001872,0.001529,...,0.004125,0.000339,0.029475,0.025339,0.000590,0.015441,5.960464e-08,0.110033,0.019129,0.033430


In [77]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_cases_countries_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [78]:
jh_deaths_countries_hist_pct_ewma = jh_deaths_countries_hist_pct.ewm(span=3).mean()

In [79]:
jh_deaths_countries_hist_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-13,0.030116,0.010938,0.012356,8.129561e-11,0.131250,8.741380e-19,0.034515,0.040190,-1.163303e-06,0.002138,...,0.004537,0.000006,0.004673,0.023649,NaN,1.192093e-07,0.0,0.093382,0.006696,3.701421e-17
2020-06-14,0.037231,0.005469,0.010783,4.064781e-11,0.065625,4.370690e-19,0.028300,0.029565,-5.816514e-07,0.001069,...,0.002700,0.000003,0.002336,0.032658,NaN,5.960464e-08,0.0,0.059191,0.053348,1.850711e-17
2020-06-15,0.026046,0.002734,0.011911,2.032390e-11,0.032813,2.185345e-19,0.026755,0.044522,-2.908257e-07,0.001273,...,0.001805,0.000001,0.001168,0.036329,NaN,2.980232e-08,0.0,0.163742,0.026674,9.253553e-18


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_deaths_countries_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [81]:
jh_recovered_countries_hist_pct_ewma = jh_recovered_countries_hist_pct.ewm(span=3).mean()

In [82]:
jh_recovered_countries_hist_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-13,0.111538,0.015464,0.014491,0.004399,0.248701,0.000013,0.043689,0.067626,0.004094,0.001992,...,0.002990,0.008770,0.030130,0.002692,0.003247,0.004925,0.166667,0.251559,0.052657,0.044537
2020-06-14,0.118135,0.010138,0.019779,0.002200,0.124350,0.000006,0.048322,0.044749,0.002998,0.002464,...,0.001495,0.006936,0.023971,0.358636,0.001623,0.002462,0.083333,0.305267,0.034481,0.022268
2020-06-15,0.105523,0.010337,0.018370,0.006221,0.086765,0.000003,0.041257,0.027363,0.001864,0.001450,...,0.001137,0.006006,0.021623,0.179318,0.000812,0.001231,0.041667,0.397916,0.026153,0.011134


In [83]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_recovered_countries_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [84]:
jh_cases_states_hist_pct_ewma = jh_cases_states_hist_pct.ewm(span=3).mean()

In [85]:
jh_cases_states_hist_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-13,0.036208,0.036571,NaN,0.049992,0.050037,0.023211,0.006293,0.005436,0.005574,5.499871e-28,...,0.018119,0.024766,0.027731,0.007732,0.000896,0.011327,0.014257,0.011536,0.013322,0.020669
2020-06-14,0.038713,0.023645,NaN,0.040086,0.041802,0.019211,0.005095,0.003763,0.004498,2.749936e-28,...,0.022495,0.021633,0.025739,0.004755,0.000448,0.011576,0.012924,0.009286,0.011990,0.015096
2020-06-15,0.032181,0.014095,NaN,0.035979,0.037540,0.020853,0.005450,0.003512,0.005951,1.374968e-28,...,0.022139,0.020351,0.023175,0.002821,0.000224,0.009274,0.012733,0.011630,0.009818,0.016510


In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_cases_states_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [87]:
jh_cases_counties_hist_pct_ewma = jh_cases_counties_hist_pct.ewm(span=3).mean()

In [88]:
jh_cases_counties_hist_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-06-13      0.033280  0.025506  0.026926  0.037612  0.077521  0.045083   
2020-06-14      0.055915  0.016908  0.031162  0.038806  0.077976  0.069643   
2020-06-15      0.037761  0.013949  0.021991  0.038634  0.057170  0.039788   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-06-13      0.031286  0.013599  0.026556  0.035281  ...  0.458333   
2020-06-14      0.034018  0.019957  0.030401  0.060193  ...  0.229167   
2020-06-15      0.021673  0.012543  0.024031  0.030097  ...  0.114583   

Province_State                                                             \
Admin2                Platte  Sheridan      Sublette Sweetwater     Teton   
2020-06-13     -6.776264e-21  0.004449  1.110223e-16   0.047106  0.006765   
2020-06-14     -3.388132e-21  0.002224  5.551115e-17   0.023553  0.008190   
2020-06-15     -1.694066e-21  0.001112  2.775558e-17   0.011776  0.004095   

Province_State                                        
Admin2             Uinta Unassigned  Washakie Weston  
2020-06-13      0.183473        NaN  0.000057    0.0  
2020-06-14      0.105821        NaN  0.000029    0.0  
2020-06-15      0.155650        NaN  0.000014    0.0  

[3 rows x 3254 columns]

In [89]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_cases_counties_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [90]:
jh_deaths_states_hist_pct_ewma = jh_deaths_states_hist_pct.ewm(span=3).mean()

In [91]:
jh_deaths_states_hist_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-13,0.011622,0.025852,NaN,0.025169,0.017715,0.016422,0.006713,0.005390,0.007969,NaN,...,0.023386,0.010647,0.019935,1.413554e-07,3.783558e-11,0.006401,0.008269,0.008173,0.007865,0.003687
2020-06-14,0.005811,0.012926,NaN,0.013425,0.014507,0.010477,0.003670,0.004487,0.007564,NaN,...,0.013812,0.008622,0.009967,7.067772e-08,1.891779e-11,0.004823,0.005784,0.004087,0.004656,0.001844
2020-06-15,0.003552,0.006463,NaN,0.011751,0.015633,0.006709,0.004025,0.002600,0.004967,NaN,...,0.015345,0.008595,0.019372,3.533886e-08,9.458895e-12,0.004352,0.004124,0.002043,0.003773,0.000922


In [92]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_deaths_states_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [93]:
jh_deaths_counties_hist_pct_ewma = jh_deaths_counties_hist_pct.ewm(span=3).mean()

In [94]:
jh_deaths_counties_hist_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga       Baldwin Barbour Bibb Blount   Bullock   
2020-06-13      0.012541  1.499897e-08     0.0  0.0    0.0  0.002604   
2020-06-14      0.006270  7.499487e-09     0.0  0.0    0.0  0.001302   
2020-06-15      0.003135  3.749744e-09     0.0  0.0    0.0  0.000651   

Province_State                                              ... Wyoming  \
Admin2            Butler       Calhoun  Chambers  Cherokee  ...    Park   
2020-06-13      0.007150  1.387779e-17  0.000011  0.063810  ...     NaN   
2020-06-14      0.003575  6.938894e-18  0.000006  0.031905  ...     NaN   
2020-06-15      0.001787  3.469447e-18  0.000003  0.015952  ...     NaN   

Province_State                                                             \
Admin2         Platte Sheridan Sublette Sweetwater Teton Uinta Unassigned   
2020-06-13        NaN      NaN      NaN        NaN   NaN   NaN   0.003918   
2020-06-14        NaN      NaN      NaN        NaN   NaN   NaN   0.001959   
2020-06-15        NaN      NaN      NaN        NaN   NaN   NaN   0.000979   

Province_State                  
Admin2         Washakie Weston  
2020-06-13          NaN    NaN  
2020-06-14          NaN    NaN  
2020-06-15          NaN    NaN  

[3 rows x 3254 columns]

In [95]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_deaths_counties_hist_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

# TODO - CONTINUE FROM HERE

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/states | [CSV](https://covidtracking.com/api/states.csv)  
States daily 4 pm ET - /api/states/daily | [CSV](http://covidtracking.com/api/states/daily.csv)  
States info - /api/states/info | [CSV](https://covidtracking.com/api/states/info.csv)  
US current - /api/us | [CSV](https://covidtracking.com/api/us.csv)  
US daily - /api/us/daily | [CSV](https://covidtracking.com/api/us/daily.csv)  
Counties- /api/counties | [CSV](https://covidtracking.com/api/counties.csv)  

In [96]:
states_curr = pd.read_csv('https://covidtracking.com/api/states.csv', index_col='state')

In [97]:
states_hist = pd.read_csv('http://covidtracking.com/api/states/daily.csv')

In [98]:
us_curr = pd.read_csv('https://covidtracking.com/api/us.csv')
us_curr

,date,states,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,lastModified,total,totalTestResults,posNeg,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,hash
0,20200615,56,2103549,21881043,1619,27115,225570,5695,9516,2636,...,2020-06-15T00:00:00Z,23986211,23984592,23984592,375,509,430967,18521,449488,1d6b3f792d9ad6ab553ce0b8f358d96d70333629


In [99]:
us_hist = pd.read_csv('https://covidtracking.com/api/us/daily.csv')

In [100]:
# change YYYYMMDD to YYYY-MM-DD and convert to datetime

states_hist['date'] = states_hist['date'].apply(lambda x : str(x)[:4] + "-" + str(x)[4:6] + "-" + str(x)[6:8])
states_hist['date'] = states_hist['date'].apply(lambda x : pd.to_datetime(x) )

In [101]:
# Now set the date as the index

states_hist.set_index('date', inplace=True)
states_hist.sort_index(inplace=True)

In [102]:
states_hist.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-01-22,WA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,2d109d8ba4c74e86fa7202b7c9f26c7617945c58,0,0,0,0,0,NaN
2020-01-23,WA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,7ed011fe93129bbc13213e7028e7e1e299d67691,0,0,0,0,0,NaN
2020-01-24,WA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,488430e6f0d933476ca2120fe0ee2ecf4aac5b94,0,0,0,0,0,NaN
2020-01-25,WA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,4a8a13ff65ef1f0bb991324d98609c765c5b5849,0,0,0,0,0,NaN
2020-01-26,WA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,5372bd69fec11b6097c548b149e4e79fca8d21d2,0,0,0,0,0,NaN


In [103]:
# get the latest date so we can pull recent data as needed
ctp_date = states_hist.index.max()
ctp_date

Timestamp('2020-06-15 00:00:00')

In [104]:
# get a list of the top 15 states using current data
top_states = states_curr['positive'].sort_values(ascending=False).nlargest(15).index
top_states

Index(['NY', 'NJ', 'CA', 'IL', 'MA', 'TX', 'PA', 'FL', 'MI', 'MD', 'GA', 'VA',
       'LA', 'CT', 'NC'],
      dtype='object', name='state')

In [105]:
# Now create a pivot tables for positive, negative, pending, hospitalized, death, total

states_positive = pd.pivot_table(states_hist, index='date', columns='state', values='positive')
states_negative = pd.pivot_table(states_hist, index='date', columns='state', values='negative')
states_pending = pd.pivot_table(states_hist, index='date', columns='state', values='pending')
states_hospitalized = pd.pivot_table(states_hist, index='date', columns='state', values='hospitalized')
states_deaths = pd.pivot_table(states_hist, index='date', columns='state', values='death')
states_tests = pd.pivot_table(states_hist, index='date', columns='state', values='total')